<a href="https://colab.research.google.com/github/OptimizationExpert/Pyomo/blob/main/yield.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install pyomo
from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd

In [24]:
N = 50
V = {i:random.randint(1,10) for i in range(1,1+N)}

In [25]:
model = AbstractModel()

M = max(V.values())
model.i = RangeSet(N)

model.u = Var(model.i, within=Binary)
model.y = Var(model.i, within=Binary)

model.min_value = Var(bounds=(0,10), within=Reals)

model.OF = Var(bounds=(0,100), within=Reals)

def rule_C1(model,i):
  return  model.min_value >= 0.7*model.u[i]*V[i]
model.C1=Constraint(model.i, rule=rule_C1)

def rule_C2(model,i):
  return  model.min_value <= model.u[i]*V[i] + M*(1-model.u[i])
model.C2=Constraint(model.i, rule=rule_C2)

def rule_C3(model):
  return  model.OF == sum(model.u[i]*V[i] for i in model.i)
model.C3=Constraint(rule=rule_C3)

def rule_C4(model,i):
  return  model.min_value - model.y[i]*V[i] <= M*(1-model.y[i])
model.C4=Constraint(model.i, rule=rule_C4)

def rule_C5(model,i):
  return  -model.min_value + model.y[i]*V[i] <= M*(1-model.y[i])
model.C5=Constraint(model.i, rule=rule_C5)

def rule_C6(model):
  return sum(model.y[i] for i in model.i )>= 1
model.C6=Constraint( rule=rule_C6)

def rule_C7(model,i):
  return  model.u[i] >= model.y[i]
model.C7=Constraint(model.i, rule=rule_C7)

model.obj1 = Objective(expr=model.OF, sense=maximize)

In [26]:
#opt = SolverFactory('cbc')
!apt-get install -y -qq coinor-cbc
opt = SolverFactory('cbc', executable='/usr/bin/cbc')
instance = model.create_instance()

In [27]:
results = opt.solve(instance) # solves and updates instance
from pyomo.opt import SolverStatus, TerminationCondition
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
     print ("this is feasible and optimal and nothing more")
elif results.solver.termination_condition == TerminationCondition.infeasible:
     print ("do something about it? or exit?")
else:
     print ('ERROR ' + str(results.solver))

print('OF= ', value(instance.obj1) )
A= [value(instance.u[i])*V[i] for i in instance.i if value(instance.u[i])>0]
print('min_value= ', value(instance.min_value) , min(A))
for i in instance.i:
  if value(instance.u[i])>0:
    print(i, value(instance.u[i]), V[i], value(instance.y[i]))

this is feasible and optimal and nothing more
OF=  100.0
min_value=  7.0 7.0
7 1.0 10 0.0
11 1.0 8 0.0
15 1.0 7 0.0
22 1.0 10 0.0
24 1.0 10 0.0
25 1.0 8 0.0
27 1.0 7 1.0
40 1.0 10 0.0
45 1.0 10 0.0
49 1.0 10 0.0
50 1.0 10 0.0
